# Consensus Simulation

In [1]:
from graph import *

In [9]:
V = tuple(range(5))
E = [(0, 1), (1, 2), (2, 3), (3, 4), (4, 0), (1, 0), (2, 1), (3, 2), (4, 3), (0, 4)]
A = np.array([[0, 1, 0, 0, 1],
              [1, 0, 1, 0, 0],
              [0, 1, 0, 1, 0],
              [0, 0, 1, 0, 1],
              [1, 0, 0, 1, 0]])
G_undir = DiGraph(V, E, A)
print("Undirected Graph:")
print("Nodes:", G_undir.graph.get_nodes())
print("Edges:", G_undir.graph.get_edges())
print("Adjacency Matrix:\n", G_undir.graph.get_adjacency_matrix())

Undirected Graph:


AttributeError: 'DiGraph' object has no attribute 'get_nodes'

{}
